In [26]:
import pandas as pd

df = pd.read_csv('/media/r2-d2/E0C494CDC494A6F6/doutorado/rec_info/data/estupro_teste.csv', sep = ';', encoding = 'utf-8')



In [27]:
df

,HISTORICO,CONTEXTO
0,Comparece neste plantão os Policiais Milit...,COTIDIANO
1,Comparece a declarante informando que viveu co...,COTIDIANO
2,"Presente o declarante, informando qu...",COTIDIANO
3,Comparece nesta especializada a senhora Tatian...,COTIDIANO
4,Comparecem nesta delegacia os policiais milita...,COTIDIANO
...,...,...
2394,Comparece neste plantão os policiais militare...,UNICO
2395,"Presente a declarante informando que, na tarde...",UNICO
2396,Comparecem os GCMs Luciane e Detz apresentando...,UNICO
2397,Comparece nesta Unidade Policial o policial mi...,UNICO


In [28]:
# baixando as libs para pre processamento

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from unidecode import unidecode


nltk.download('stopwords')

stopwords = set(stopwords.words('portuguese'))

stemmer = PorterStemmer()
#nltk.download("wordnet")
#from nltk.stem import WordNetLemmatizer
#lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /home/r2-d2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
# pre processamento

    # Remover caracteres especiais e números
    #text = re.sub('[^A-Za-z\s\']+', '', text)

def preprocess_text(text):

    padrao_arroba = re.compile(r'@\S+')
      # Remover as strings com "@" no início da string
    text = re.sub(padrao_arroba, '', text)

        #remover sites
    #pattern_url = re.compile(r'https?:\/\/.*?[\s+]')

    text = re.sub(r'http\S+', '', text)
    

    # Converter tudo para letras minúsculas
    text = text.lower()

    
    # Remover as tags HTML
    text = re.sub('<[^<]+?>', '', text)    

    text = unidecode(text)
    
    
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # remover palavras que so possua caractere especial
    words = [word for word in words if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words = [word for word in words if len(word) > 2]

    
    # Remover as stopwords
    words = [word for word in words if word not in stopwords]
    
    # Aplicar o stemming
    words = [stemmer.stem(word) for word in words]
    #words = [lemmatizer.lemmatize(word) for word in words]

    # Juntar as palavras em uma única string
    text = ' '.join(words)
  
    return text

In [30]:
# rodar o pre processamento

def preprocess_collection(docs):

    preprocessed_documents = []

    #for document in docs:
    for index, row in docs.iterrows():
        tema = row['CONTEXTO']
        #nome_arquivo = document['nome_arquivo']
        conteudo = row['HISTORICO']
        
        preprocessed_conteudo = preprocess_text(conteudo)
        
        preprocessed_document = {
            'tema': tema,
        #    'nome_arquivo': nome_arquivo,
            'conteudo': preprocessed_conteudo
        }
        
        preprocessed_documents.append(preprocessed_document)
        
    return preprocessed_documents

In [31]:
df.head(4)

,HISTORICO,CONTEXTO
0,Comparece neste plantão os Policiais Milit...,COTIDIANO
1,Comparece a declarante informando que viveu co...,COTIDIANO
2,"Presente o declarante, informando qu...",COTIDIANO
3,Comparece nesta especializada a senhora Tatian...,COTIDIANO


In [32]:
df = df.sort_values('CONTEXTO')

In [47]:
preprocessed_documents = preprocess_collection(df)

In [46]:
len(preprocessed_documents)

1500

In [48]:
frases = [o['conteudo'] for o in preprocessed_documents]

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
# vetorizando

#temas = [doc["tema"] for doc in preprocessed_documents]
temas = []
for doc in preprocessed_documents:
    temas.append(doc['tema'])


conteudos = [doc["conteudo"] for doc in preprocessed_documents]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(conteudos)

In [51]:
preprocessed_documents[10]

{'tema': 'COTIDIANO',
 'conteudo': 'comparecem nesta delegacia policia conselheira tutelar claudia ligia noticiando caso suposto abuso crianca estupro vulneravel fazendo vitima crianca qualificada sete ano idad conselheira informam tomaram conhecimento crianca acima sido supostament abusada sexualment padrasto cicero pereira santo testemunha acima arrolada relataram conhecem vizinho sandra campo elia cicero pereira santo bem filha sandra crianca raissa elia rubio alvejaniea sete ano idad esclarecendo raissa amiguinha filha ana leticia ano idad frequencia brincam junta sexta-feira dia 16/04/21 crianca raissa ate casa brincar ana leticia ond deve ter chegado volta 18h30 afirmaram certo momento durant jantar avisaram raissa iriam leva-la embora momento ana leticia afirm raissa nao queria embora porqu padrasto dede cicero fazia mal assim conversa sobr assunto crianca acab relatando dede passava mao regiao vagin relatando fato tambem meio gesto testemunha disseram crianca relat fato maneira

In [38]:
    input_tfidf = vectorizer.transform([preprocessed_documents[10]['conteudo']])
    similaridades = cosine_similarity(input_tfidf, tfidf_matrix)[0]

    resultados = list(zip(similaridades, temas, conteudos))
    resultados.sort(reverse=True)

In [39]:
len(temas)

1500

In [40]:
len(conteudos)

1500

In [3]:
#dataset de teste

import pandas as pd

df_test = pd.read_csv('data/Corona_NLP_test.csv', encoding = "ISO-8859-1")
df_test_new = df_test[['OriginalTweet', 'Sentiment']]

In [4]:
df_test_new

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral
...,...,...
3793,Meanwhile In A Supermarket in Israel -- People...,Positive
3794,Did you panic buy a lot of non-perishable item...,Negative
3795,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral
3796,Gov need to do somethings instead of biar je r...,Extremely Negative


In [65]:
df_test_sorted = df_test_new.sort_values('Sentiment')

In [66]:
doc_consultas_preproc = preprocess_collection(df_test_sorted)

In [32]:
doc_consultas_preproc[600]

{'tema': 'Extremely Positive',
 'conteudo': 'grocery store worker also front lines- thankful need protect coronavirus protecting employee serving absolutely necessary role society'}

In [52]:
array_temas = ['COTIDIANO',
'EVENTUAL',
'UNICO']
result_final = {}

for d in array_temas:
    result_final[d] = {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}

print(result_final)    

{'COTIDIANO': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'EVENTUAL': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'UNICO': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}}


In [53]:
def calcula_map(resultados, tema_verificado):
    acertos = 0
    maps = []
    #for i, doc in enumerate(ranking):
    for i, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
        if tema_verificado == tema:
            acertos +=1
            maps.append(acertos/(i+1))
    return mean(maps)

In [54]:
from statistics import mean

#input_documento = doc_consultas_preproc[150]['conteudo']

p10 = []
p20 = []
p50 = []
p100 = []
map_f = []
tema_atual = ""
for c, input_documento in enumerate(preprocessed_documents):
    #print(c)
    if c == 0:
        tema_atual = input_documento['tema']
    #print(tema_atual)
    acertos = 0     

    input_tfidf = vectorizer.transform([input_documento['conteudo']])
    similaridades = cosine_similarity(input_tfidf, tfidf_matrix)[0]

    resultados = list(zip(similaridades, temas, conteudos))
    resultados.sort(reverse=True)
    #print(len(resultados))
    map_f.append(calcula_map(resultados, input_documento['tema']))
    for rank, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
        #print(rank)
        if input_documento['tema'] == tema:
            #print("acertou")
            acertos += 1
            
        if rank == 10:
            #print("deu 10")
            p10.append(acertos/10.0)
            #print(p10)
        if rank == 20:
            p20.append(acertos/20.0)
        if rank == 50:
            p50.append(acertos/50.0)
        if rank == 100:
            p100.append(acertos/100.0)
        
    
    if tema_atual != input_documento['tema'] or c == len(preprocessed_documents) - 1:
        print("trocou")
#        print(p10)
        result_final[tema_atual]['p10'] = mean(p10)
        result_final[tema_atual]['p20'] = mean(p20)
        result_final[tema_atual]['p50'] = mean(p50)
        result_final[tema_atual]['p100'] = mean(p100)     
        result_final[tema_atual]['map'] = mean(map_f)
        p10 = []
        p20 = []
        p50 = []
        p100 = []
        map_f = []           

    tema_atual = input_documento['tema']        


trocou
trocou
trocou


In [55]:
result_final

{'COTIDIANO': {'p10': 0.550936329588015,
  'p20': 0.5270287141073658,
  'p50': 0.5040948813982522,
  'p100': 0.4893882646691636,
  'map': 0.4027031716837907},
 'EVENTUAL': {'p10': 0.46332916145181474,
  'p20': 0.4326032540675845,
  'p50': 0.4065081351689612,
  'p100': 0.39296620775969965,
  'map': 0.35909269850215414},
 'UNICO': {'p10': 0.5387984981226533,
  'p20': 0.5143929912390488,
  'p50': 0.48630788485607007,
  'p100': 0.46759699624530665,
  'map': 0.41316800737152065}}

In [56]:
import json
with open('/media/r2-d2/E0C494CDC494A6F6/doutorado/rec_info/results/est/est_tfidf_menor.json', 'w') as f:
    json.dump(result_final, f)

In [57]:
import pandas as pd

df = pd.DataFrame(result_final)

In [58]:
df =df.transpose()

In [59]:
df

,p10,p20,p50,p100,map
COTIDIANO,0.550936,0.527029,0.504095,0.489388,0.402703
EVENTUAL,0.463329,0.432603,0.406508,0.392966,0.359093
UNICO,0.538798,0.514393,0.486308,0.467597,0.413168


In [60]:
df.describe()

,p10,p20,p50,p100,map
count,3.000000,3.000000,3.000000,3.000000,3.000000
mean,0.517688,0.491342,0.465637,0.449984,0.391655
std,0.047466,0.051260,0.051974,0.050567,0.028681
min,0.463329,0.432603,0.406508,0.392966,0.359093
25%,0.501064,0.473498,0.446408,0.430282,0.380898
50%,0.538798,0.514393,0.486308,0.467597,0.402703
75%,0.544867,0.520711,0.495201,0.478493,0.407936
max,0.550936,0.527029,0.504095,0.489388,0.413168
